In [26]:
src_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
src_zip = "dakshina_dataset_v1.0.tar"
DATA_SRC="dakshina_dataset_v1.0/ta/lexicons"
DATA_TRAIN_SRC = "/ta.translit.sampled.train.tsv" 
DATA_TEST_SRC = "/ta.translit.sampled.test.tsv"
#TRAIN_IMAGES_PER_LABEL = 1000
#TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}
PROJECT_NAME = "CS6910 ASSIGNMENT 3"
dataset='dakshina-dataset'

In [5]:
%%capture
!curl -SL $src_url > $src_zip
!tar -xf $src_zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

Uploading Data: only run once

In [ ]:

run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(dataset, type="raw_data")

raw_data_at.add_dir(DATA_SRC)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

Downloading Data

In [23]:
run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="download")

# Query W&B for an artifact and mark it as input to this run
artifact = run.use_artifact(dataset+':latest')

# Download the artifact's contents
artifact_dir = artifact.download()
run.finish()

In [27]:
colnames=["ntv","rmn",'nAtt']
df_train = pd.read_csv(artifact_dir + DATA_TRAIN_SRC,sep="\t",names=colnames,na_filter=False)


In [5]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [30]:
input_texts = df_train.ntv.to_list()
target_texts = df_train.rmn.apply(lambda s:'\t'+s+'\n').to_list()
input_characters = set(df_train.ntv.sum())
target_characters = set(df_train.rmn.sum())


input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [31]:
target_texts

['\tfiat\n',
 '\tphiyat\n',
 '\tpiyat\n',
 '\tfiraans\n',
 '\tfrance\n',
 '\tfrancis\n',
 '\tpiraance\n',
 '\tfilim\n',
 '\tfilm\n',
 '\tpilim\n',
 '\tfilims\n',
 '\tfilms\n',
 '\tpilims\n',
 '\tfacebook\n',
 '\thpaespuk\n',
 '\tpaespuk\n',
 '\tfoan\n',
 '\thpoan\n',
 '\tphone\n',
 '\tpoan\n',
 '\tforce\n',
 '\thpoars\n',
 '\tphoars\n',
 '\tpoars\n',
 '\tpores\n',
 '\tagrinaip\n',
 '\tahrinaip\n',
 '\tarinaip\n',
 '\taga\n',
 '\taka\n',
 '\tagachchivappu\n',
 '\takachchivappu\n',
 '\tagathi\n',
 '\takathi\n',
 '\tagathigalin\n',
 '\takathikalin\n',
 '\tagathigalukkaana\n',
 '\tagathikalukkaana\n',
 '\tagathigalai\n',
 '\tagathigal\n',
 '\takathigal\n',
 '\tagathiyaaga\n',
 '\takathiyaaka\n',
 '\tagaththinai\n',
 '\takaththinai\n',
 '\tagaththiyam\n',
 '\takaththiyam\n',
 '\tagaththiyar\n',
 '\takaththiyar\n',
 '\tagatheeswarar\n',
 '\tagaththeesvarar\n',
 '\tagaththeeswarar\n',
 '\takaththeesvarar\n',
 '\taganaanooru\n',
 '\tagananuru\n',
 '\takanaanooru\n',
 '\taganthaiyae\n',
 '\taga

In [29]:
input_texts

['ஃபியட்',
 'ஃபியட்',
 'ஃபியட்',
 'ஃபிரான்ஸ்',
 'ஃபிரான்ஸ்',
 'ஃபிரான்ஸ்',
 'ஃபிரான்ஸ்',
 'ஃபிலிம்',
 'ஃபிலிம்',
 'ஃபிலிம்',
 'ஃபிலிம்ஸ்',
 'ஃபிலிம்ஸ்',
 'ஃபிலிம்ஸ்',
 'ஃபேஸ்புக்',
 'ஃபேஸ்புக்',
 'ஃபேஸ்புக்',
 'ஃபோன்',
 'ஃபோன்',
 'ஃபோன்',
 'ஃபோன்',
 'ஃபோர்ஸ்',
 'ஃபோர்ஸ்',
 'ஃபோர்ஸ்',
 'ஃபோர்ஸ்',
 'ஃபோர்ஸ்',
 'அஃறிணைப்',
 'அஃறிணைப்',
 'அஃறிணைப்',
 'அக',
 'அக',
 'அகச்சிவப்பு',
 'அகச்சிவப்பு',
 'அகதி',
 'அகதி',
 'அகதிகளின்',
 'அகதிகளின்',
 'அகதிகளுக்கான',
 'அகதிகளுக்கான',
 'அகதிகளை',
 'அகதிகள்',
 'அகதிகள்',
 'அகதியாக',
 'அகதியாக',
 'அகத்திணை',
 'அகத்திணை',
 'அகத்தியம்',
 'அகத்தியம்',
 'அகத்தியர்',
 'அகத்தியர்',
 'அகத்தீஸ்வரர்',
 'அகத்தீஸ்வரர்',
 'அகத்தீஸ்வரர்',
 'அகத்தீஸ்வரர்',
 'அகநானூறு',
 'அகநானூறு',
 'அகநானூறு',
 'அகந்தையை',
 'அகந்தையை',
 'அகந்தையை',
 'அகந்தையை',
 'அகன்ற',
 'அகன்ற',
 'அகன்று',
 'அகன்று',
 'அகன்றும்',
 'அகன்றும்',
 'அகப்பொருள்',
 'அகப்பொருள்',
 'அகமது',
 'அகமது',
 'அகமது',
 'அகமது',
 'அகம்',
 'அகம்',
 'அகர',
 'அகர',
 'அகரம்',
 'அகரம்',
 'அகராதி',
 'அகராதி',
 'அகராதிகள்